In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

pd.set_option('display.max_columns', None)

%load_ext autoreload
%autoreload 2

In [3]:
target = pd.read_csv("../data/target_train.csv")
sample = pd.read_csv("../data/sample_submission.csv")

plavki_train = pd.read_csv("../data/plavki_train.csv")
plavki_test = pd.read_csv("../data/plavki_test.csv")

gas_train = pd.read_csv("../data/gas_train.csv")
gas_test = pd.read_csv("../data/gas_test.csv")

chugun_train = pd.read_csv("../data/chugun_train.csv")
chugun_test = pd.read_csv("../data/chugun_test.csv")

lom_train = pd.read_csv("../data/lom_train.csv")
lom_test = pd.read_csv("../data/lom_test.csv")

produv_train = pd.read_csv("../data/produv_train.csv")
produv_test = pd.read_csv("../data/produv_test.csv")

chronom_train = pd.read_csv("../data/chronom_train.csv")
chronom_test = pd.read_csv("../data/chronom_test.csv")

sip_train = pd.read_csv("../data/sip_train.csv")
sip_test = pd.read_csv("../data/sip_test.csv")


In [85]:
s = chronom_train.groupby("NPLV")["NOP"].apply(set).values[0]
for s1 in chronom_train.groupby("NPLV")["NOP"].apply(set).values:
    s = s.intersection(s1)

In [86]:
s

{'Выпуск металла',
 'Завалка лома',
 'Заливка чугуна',
 'Повалка',
 'Продувка',
 'Слив шлака',
 'межпл. простой'}

In [87]:
s = chronom_test.groupby("NPLV")["NOP"].apply(set).values[0]
for s1 in chronom_test.groupby("NPLV")["NOP"].apply(set).values:
    s = s.intersection(s1)

In [4]:
s = {'Завалка лома', 'Продувка', 'межпл. простой'}

In [14]:
trains = []
tests = []

for event in s:
    for timestamp in ["VR_NACH", "VR_KON"]:
        kon_step_train = chronom_train[chronom_train["NOP"] == event].set_index("NPLV")[timestamp]
        kon_step_test = chronom_test[chronom_test["NOP"] == event].set_index("NPLV")[timestamp]
        
        gas_train_cp = gas_train.join(kon_step_train, on="NPLV")
        gas_test_cp = gas_test.join(kon_step_test, on="NPLV")
        
        gas_train_cp["delta"] = (pd.to_datetime(gas_train_cp[timestamp]) - pd.to_datetime(gas_train_cp["Time"])).dt.total_seconds()
        gas_test_cp["delta"] = (pd.to_datetime(gas_test_cp[timestamp]) - pd.to_datetime(gas_test_cp["Time"])).dt.total_seconds()

        gas_train_cp.loc[gas_train_cp["delta"] > 0, "delta"] = 1e9
        gas_test_cp.loc[gas_test_cp["delta"] > 0, "delta"] = 1e9
        
        train_delta = gas_train_cp.groupby("NPLV")["delta"].apply(lambda df:df.idxmin())
        test_delta = gas_test_cp.groupby("NPLV")["delta"].apply(lambda df:df.idxmin())
        
        res_train = gas_train_cp.iloc[train_delta].set_index("NPLV").drop(columns=["Time", timestamp])
        res_test = gas_test_cp.iloc[test_delta].set_index("NPLV").drop(columns=["Time", timestamp])
        
        res_train.columns = [f"{event}_{timestamp}_" + el for el in res_train.columns]
        res_test.columns = [f"{event}_{timestamp}_" + el for el in res_test.columns]
        
        trains.append(res_train)
        tests.append(res_test)
        print("Ы")

Ы
Ы
Ы
Ы
Ы
Ы
Ы
Ы


In [23]:
trains[]

,Заливка чугуна_VR_KON_V,Заливка чугуна_VR_KON_T,Заливка чугуна_VR_KON_O2,Заливка чугуна_VR_KON_N2,Заливка чугуна_VR_KON_H2,Заливка чугуна_VR_KON_CO2,Заливка чугуна_VR_KON_CO,Заливка чугуна_VR_KON_AR,Заливка чугуна_VR_KON_T фурмы 1,Заливка чугуна_VR_KON_T фурмы 2,Заливка чугуна_VR_KON_O2_pressure,Заливка чугуна_VR_KON_delta
NPLV,,,,,,,,,,,,
510008,219321.343750,131.944443,9.966543,78.383922,0.010667,10.125831,5.471810e-01,0.973502,0.000000,0.000000,13.389757,-1965.437
510009,218792.984375,114.236115,6.053720,77.350703,0.006827,14.667046,9.285339e-01,0.983698,0.000000,0.000000,13.628472,-2432.437
510010,216986.937500,127.430550,8.751750,79.672749,0.020500,9.889875,6.478750e-01,1.007250,0.000000,0.000000,13.823785,-2144.437
510011,221631.140625,112.847221,12.240000,75.919998,0.000000,10.420000,4.000000e-01,1.030000,0.000000,0.000000,14.626736,-2687.437
510012,219532.343750,133.333328,7.720000,75.199997,0.000000,15.760000,3.400000e-01,0.980000,0.000000,0.000000,15.567130,-2154.437
...,...,...,...,...,...,...,...,...,...,...,...,...
512318,206042.953125,131.597214,12.310000,77.099998,0.000000,8.840000,7.600000e-01,0.980000,24.276621,26.060578,16.109484,-2739.437
512319,213545.890625,134.195474,13.940000,78.160004,0.000000,6.360000,5.100000e-01,1.030000,24.019264,26.042221,14.351852,-2203.437
512320,206379.718750,123.611115,11.090000,76.400002,0.000000,10.930000,6.200000e-01,0.960000,24.645240,26.475694,15.509260,-2456.437


In [45]:
final_train = trains[0]
for i in range(1, len(trains)):
    final_train = final_train.join(trains[i], on="NPLV")

In [28]:
final_train = final_train.dropna(axis=1)

In [40]:
final_test = tests[0]
for i in range(1, len(tests)):
    final_test = final_test.join(tests[i], on="NPLV")

In [31]:
final_test = final_test[final_train.columns]

In [36]:
final_train.to_csv("final_train.csv", index=True)
final_test.to_csv("final_test.csv", index=True)

In [49]:
for el in trains:
    print(el.index.nunique())

2063
2063
1003
1003
2063
2063
2063
2063


In [72]:
gas_train = gas_train.join(kon_step_train, on="NPLV")
gas_test = gas_test.join(kon_step_test, on="NPLV")

In [73]:
gas_train["delta"] = (pd.to_datetime(gas_train["VR_NACH"]) - pd.to_datetime(gas_train["Time"])).dt.total_seconds()
gas_test["delta"] = (pd.to_datetime(gas_test["VR_NACH"]) - pd.to_datetime(gas_test["Time"])).dt.total_seconds()

gas_train.loc[gas_train["delta"] > 0, "delta"] = 1e9
gas_test.loc[gas_test["delta"] > 0, "delta"] = 1e9

In [ ]:
gas_train

In [74]:
train_delta = gas_train.groupby("NPLV")["delta"].apply(lambda df:df.idxmin())
test_delta = gas_test.groupby("NPLV")["delta"].apply(lambda df:df.idxmin())

In [75]:
res_train = gas_train.iloc[train_delta].set_index("NPLV").drop(columns=["Time", "VR_NACH"])
res_test = gas_test.iloc[test_delta].set_index("NPLV").drop(columns=["Time", "VR_NACH"])

In [76]:
res_train.columns = ["продувка_res2_" + el for el in res_train.columns]
res_test.columns = ["продувка_res2_" + el for el in res_test.columns]

In [77]:
res_train.to_csv("res2_train.csv")
res_test.to_csv("res2_test.csv")

In [40]:
gas_test.iloc[test_delta]

IndexError: positional indexers are out-of-bounds

In [26]:
train_delta

NPLV
510008    0.563
510009    0.563
510010    0.563
510011    0.563
510012    0.563
          ...  
512318    1.563
512319    0.563
512320    0.563
512321    0.563
512322    0.563
Name: delta, Length: 2063, dtype: float64

In [30]:
gtrain["delta"]

NPLV
510008    2.518563e+03
510008    2.517563e+03
510008    2.516563e+03
510008    2.515563e+03
510008    2.514563e+03
              ...     
512322    1.000000e+09
512322    1.000000e+09
512322    1.000000e+09
512322    1.000000e+09
512322    1.000000e+09
Name: delta, Length: 6473642, dtype: float64

In [31]:
train_delta

NPLV
510008    0.563
510009    0.563
510010    0.563
510011    0.563
510012    0.563
          ...  
512318    1.563
512319    0.563
512320    0.563
512321    0.563
512322    0.563
Name: delta, Length: 2063, dtype: float64

In [29]:
gtrain = gas_train.set_index("NPLV").loc[train_delta.index]
gtrain = gtrain[gtrain["delta"] == train_delta]

ValueError: Can only compare identically-labeled Series objects

In [13]:
gas_train.groupby

,NPLV,Time,V,T,O2,N2,H2,CO2,CO,AR,T фурмы 1,T фурмы 2,O2_pressure,VR_KON,delta
0,510008,2021-01-01 03:08:11.437,218263.343750,262.847229,18.722993,80.132247,0.087755,0.163878,9.229025e-03,0.893243,0.000000,0.000000,13.085938,2021-01-01 03:50:10,-1 days +23:18:01.437000
1,510008,2021-01-01 03:08:12.437,218263.343750,262.847229,18.732721,80.138406,0.087959,0.148980,8.390023e-03,0.892948,0.000000,0.000000,13.085938,2021-01-01 03:50:10,-1 days +23:18:02.437000
2,510008,2021-01-01 03:08:13.437,218369.359375,262.152771,18.742449,80.144565,0.088163,0.134082,7.551021e-03,0.892653,0.000000,0.000000,13.085938,2021-01-01 03:50:10,-1 days +23:18:03.437000
3,510008,2021-01-01 03:08:14.437,218475.359375,261.805573,18.752177,80.150724,0.088367,0.119184,6.712018e-03,0.892358,0.000000,0.000000,13.093172,2021-01-01 03:50:10,-1 days +23:18:04.437000
4,510008,2021-01-01 03:08:15.437,218369.359375,260.763885,18.761905,80.156883,0.088571,0.104286,5.873016e-03,0.892063,0.000000,0.000000,13.093172,2021-01-01 03:50:10,-1 days +23:18:05.437000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6468013,512322,2021-04-26 18:48:35.437,207386.734375,118.402779,1.890000,97.220001,0.090000,0.050000,0.000000e+00,0.750000,23.900463,27.054343,14.424190,2021-04-26 18:47:40,0 days 00:00:55.437000
6468014,512322,2021-04-26 18:48:36.437,207051.609375,118.402779,2.710000,96.410004,0.080000,0.040000,0.000000e+00,0.770000,23.900463,27.054342,14.424190,2021-04-26 18:47:40,0 days 00:00:56.437000
6468015,512322,2021-04-26 18:48:37.437,207163.375000,118.402779,3.770000,95.349998,0.090000,0.030000,0.000000e+00,0.770000,23.900463,27.054341,14.438658,2021-04-26 18:47:40,0 days 00:00:57.437000
6468016,512322,2021-04-26 18:48:38.437,206827.906250,118.750000,3.400000,95.720001,0.090000,0.040000,0.000000e+00,0.750000,23.900463,27.054340,14.445890,2021-04-26 18:47:40,0 days 00:00:58.437000


In [18]:
gas_train["NPLV"].map()

0          510008
1          510008
2          510008
3          510008
4          510008
            ...  
6468013    512322
6468014    512322
6468015    512322
6468016    512322
6468017    512322
Name: NPLV, Length: 6468018, dtype: int64